Importar as bibliotecas necessárias

In [1]:
from pysus.ftp.databases.sih import SIH

import pandas as pd
import os

Carregar a base do "Sistema de Internações Hospitalares" (SIH)

In [2]:
sih = SIH().load()

Declarar um array dos arquivos de dados do registro de "Autorização de Internação Hospitalar - Dados Reduzidos" - um para cada ano

In [3]:
files_2019 = sih.get_files('RD', uf='RS', year=2019)
files_2020 = sih.get_files('RD', uf='RS', year=2020)

Definir os caminhos dos .parquet

In [4]:
path_raw_2019 = './data/raw/parquet/2019/'
path_raw_2020 = './data/raw/parquet/2020/'

Baixar todos os .parquet de cada ano

In [5]:
# sih.download(files_2019, local_dir=path_raw_2019)
# sih.download(files_2020, local_dir=path_raw_2020)

Iterar sobre cada arquivo .parquet e juntar os anos respectivos em datasets combinados

In [6]:
parquet_files_2019 = [file for file in os.listdir(path_raw_2019) if file.endswith('.parquet')]
parquet_files_2020 = [file for file in os.listdir(path_raw_2020) if file.endswith('.parquet')]

dfs_2019 = []
dfs_2020 = []

for file in parquet_files_2019:
    df = pd.read_parquet(os.path.join(path_raw_2019, file))
    dfs_2019.append(df)

for file in parquet_files_2020:
    df = pd.read_parquet(os.path.join(path_raw_2020, file))
    dfs_2020.append(df)

dataframe_2019 = pd.concat(dfs_2019, ignore_index=True)
dataframe_2020 = pd.concat(dfs_2020, ignore_index=True)

Definir o dicionários que serão utilizados para tornar os dados mais legíveis e listas que irão servir como máscaras de colunas que vamos manter e descartar

In [7]:
dtypes = {
    'cod': str,
    'value': str
}

espec = pd.read_csv("data/parameters/csv/LEITOS.csv", dtype={'cod': str,'value': str})
cid = pd.read_csv("data/parameters/csv/CID10-TABLE.csv")

ESPEC = espec.set_index('cod')['value'].to_dict()
CID = cid.set_index('SUBCAT')['DESCRICAO'].to_dict()

UTI_MES_TO = {
"000":"Não",
"001":"Sim",
"999":"Sim"}

COBRANCA = {
'11':"Alta curado",
'12':"Alta melhorado",
'14':"Alta a pedido",
'15':"Alta com previsão de retorno p/acomp do paciente",
'16':"Alta por evasão",
'18':"Alta por outros motivos",
'19':"Alta de paciente agudo em psiquiatria",
'21':"Permanência por características próprias da doença",
'22':"Permanência por intercorrência",
'23':"Permanência por impossibilidade sócio-familiar",
'24':"Permanência proc doação órg, tec, cél-doador vivo",
'25':"Permanência proc doação órg, tec, cél-doador morto",
'26':"Permanência por mudança de procedimento",
'27':"Permanência por reoperação",
'28':"Permanência por outros motivos",
'29':"Transferência para internação domiciliar",
'32':"Transferência para internação domiciliar",
'31':"Transferência para outro estabelecimento",
'41':"Óbito com DO fornecida pelo médico assistente",
'42':"Óbito com DO fornecida pelo IML",
'43':"Óbito com DO fornecida pelo SVO",
'51':"Encerramento administrativo",
'61':"Alta da mãe/puérpera e do recém-nascido",
'17':"Alta da mãe/puérpera e do recém-nascido",
'62':"Alta da mãe/puérpera e permanência recém-nascido",
'13':"Alta da mãe/puérpera e permanência recém-nascido",
'63':"Alta da mãe/puérpera e óbito do recém-nascido",
'64':"Alta da mãe/puérpera com óbito fetal",
'65':"Óbito da gestante e do concepto",
'66':"Óbito da mãe/puérpera e alta do recém-nascido",
'67':"Óbito da mãe/puérpera e permanência recém-nascido"
}

COD_IDADE = {
'2':"Dias",
'3':"Meses",
'4':"Anos",
'0':""}

SEXO = {
'1':"Masculino",
'2':"Feminino",
'3':"Feminino",
'0':"",
'9':""
}

RACA_COR = {
"01":"Branca",
"02":"Preta",
"03":"Parda",
"04":"Amarela",
"05":"Indígena",
"99":"Sem Informação"
}

DIAS_PERM = {
"0000":"0 dias",
"0001":"1 dia",
"0002":"2 dias",
"0003":"3 dias",
"0004":"4 dias",
"0005":"5 dias",
"0006":"6 dias",
"0007":"7 dias", 
"0014":"8-14 dias",
"0021":"15-21 dias",
"0028":"22-28 dias", 
'9999':"29 dias e +"
}

MORTE = {
'0':"Sem óbito",
'1':"Com óbito"
}

INSTRU = {
'1':"Analfabeto",
'2':"1º grau",
'3':"2º grau",
'4':"3º grau",
'0':"",
'9':""
}

MARCA_UTI = {
'00':"Não utilizou UTI",
'74':"UTI adulto - tipo I",
'75':"UTI adulto - tipo II",
'76':"UTI adulto - tipo III",
'77':"UTI infantil - tipo I",
'78':"UTI infantil - tipo II",
'79':"UTI infantil - tipo III",
'80':"UTI neonatal - tipo I", 
'81':"UTI neonatal - tipo II", 
'82':"UTI neonatal - tipo III", 
'83':"UTI de queimados",
'85':"UTI coronariana tipo II - UCO tipo II",
'86':"UTI coronariana tipo III - UCO tipo III",
'99':"UTI Doador", "01":"Utilizou mais de um tipo de UTI"
}

MARCA_UCI = {
"00":"Não utilizou UCI",
"01":"Unidade de cuidados intermed neonatal convencional",
"02":"Unidade de cuidados intermed neonatal canguru",
"03":"Unidade intermediária neonatal"
}

TPDISEC = {
'1':"Pré-existente",
'2':"Adquirido",
'0':""
}

unecessary_columns = [
"CID_NOTIF",
"INFEHOSP",
"ANO_CMPT",
"UF_ZI",
"MES_CMPT",
"CGC_HOSP",
"ETNIA",
"N_AIH",
"IDENT",
"CEP",
"NASC",
"UTI_MES_IN",
"UTI_MES_AN",
"UTI_MES_AL",
"UTI_INT_IN",
"UTI_INT_AN",
"UTI_INT_AL",
"UTI_INT_TO",
"DIAR_ACOM",
"PROC_SOLIC",
"PROC_REA",
"VAL_SH",
"VAL_SP",
"VAL_SADT",
"VAL_RN",
"VAL_ACOMP",
"VAL_ORTP",
"VAL_SANGUE",
"VAL_SADTSR",
"VAL_TRANSP",
"VAL_OBSANG",
"VAL_PED1AC",
"VAL_TOT",
"VAL_UTI",
"US_TOT",
"NATUREZA",
"NAT_JUR",
"GESTAO",
"RUBRICA",
"IND_VDRL",
"MUNIC_MOV",
"NACIONAL",
"NUM_PROC",
"CAR_INT",
"TOT_PT_SP",
"CPF_AUT",
"HOMONIMO",
"CONTRACEP1",
"CONTRACEP2",
"GESTRISCO",
"INSC_PN",
"SEQ_AIH5",
"CBOR",
"CNAER",
"VINCPREV",
"GESTOR_COD",
"GESTOR_TP",
"GESTOR_CPF",
"GESTOR_DT",
"CNES",
"CNPJ_MANT",
"COMPLEX",
"FINANC",
"FAEC_TP",
"REGCT",
"SEQUENCIA",
"REMESSA",
"AUD_JUST",
"SIS_JUST",
"VAL_SH_FED",
"VAL_SP_FED",
"VAL_SH_GES",
"VAL_SP_GES",
"VAL_UCI"
]

necessary_columns = [
"QT_DIARIAS", 
"DT_INTER",
"DT_SAIDA",
"DIAG_PRINC",
"COBRANCA",
"IDADE",
"SEXO",
"DIAS_PERM", 
"MORTE"
]

format_columns = {
"UTI_MES_TO": "Dias_na_UTI",
"UF_ZI": "Municipio", ######
"ESPEC": "Especialidade",
"MUNIC_RES": "Municipio_do_paciente", #####
"QT_DIARIAS": "Diarias",
"DT_INTER": "Data_de_internacao",
"DT_SAIDA": "Data_de_alta",
"DIAG_PRINC": "Diagnostico",
"DIAG_SECUN": "Diagnostico_Secundario",
"COBRANCA": "Motivo_Saida_Permanencia",
"COD_IDADE": "Formato_Idade",
"IDADE": "Idade",
"SEXO": "Sexo",
"RACA_COR": "Cor",
"DIAS_PERM": "Permanencia",
"MORTE": "Morte",
"NUM_FILHOS": "Filhos",
"INSTRU": "Grau_Escolaridade",
"CID_NOTIF": "CID10_Notificacao",
"INFEHOSP": "Infeccao_Hospitalar",
"CID_MORTE": "CID10_Morte",
"MARCA_UTI": "Tipo_UTI",
"MARCA_UCI": "Tipo_UCI ",
"DIAGSEC1": "Diagnostico_Secundario_1",
"DIAGSEC2": "Diagnostico_Secundario_2",
"DIAGSEC3": "Diagnostico_Secundario_3",
"DIAGSEC4": "Diagnostico_Secundario_4",
"DIAGSEC5": "Diagnostico_Secundario_5",
"DIAGSEC6": "Diagnostico_Secundario_6",
"DIAGSEC7": "Diagnostico_Secundario_7",
"DIAGSEC8": "Diagnostico_Secundario_8",
"DIAGSEC9": "Diagnostico_Secundario_9",
"TPDISEC1": "Tipo_Diagnostico_1",
"TPDISEC2": "Tipo_Diagnostico_2",
"TPDISEC3": "Tipo_Diagnostico_3",
"TPDISEC4": "Tipo_Diagnostico_4",
"TPDISEC5": "Tipo_Diagnostico_5",
"TPDISEC6": "Tipo_Diagnostico_6",
"TPDISEC7": "Tipo_Diagnostico_7",
"TPDISEC8": "Tipo_Diagnostico_8",
"TPDISEC9": "Tipo_Diagnostico_9"
}

Atribuir o dataframe desejado à um dataframe temporário para fazer a limpeza, além de definir o caminho pelo qual o dataset limpo será exportado.

In [8]:
df = dataframe_2020
year = '2020'
output_path = './data/cleaned/csv/' + year + '/output_' + year + '.csv'

Substituir os valores em branco por "None" utilizando regex


In [9]:
df = df.replace(r'^\s*$', None, regex=True)

Descartar as seguintes colunas/linhas:
- Linhas cujas colunas necessárias são vazias
- Colunas desnecessárias
- Dados duplicados
- Colunas onde todas as linhas são "None" ou "0"s

In [10]:
df = df.dropna(subset=necessary_columns)

df = df.drop(columns=unecessary_columns)

df = df.drop_duplicates(df)

df = df.drop(columns=df.columns[(df.isna()).all()])

df = df.drop(columns=df.columns[((df == "0")).all()])

df = df.drop(columns=df.columns[((df == "00")).all()])

df = df.drop(columns=df.columns[((df == "000")).all()])

df = df.drop(columns=df.columns[((df == "0000")).all()])

df = df.drop(columns=df.columns[((df == "00000")).all()])

Mostrar as colunas que ainda existem no dataframe

In [11]:
df.columns

Index(['ESPEC', 'MUNIC_RES', 'SEXO', 'UTI_MES_TO', 'MARCA_UTI', 'QT_DIARIAS',
       'DT_INTER', 'DT_SAIDA', 'DIAG_PRINC', 'COBRANCA', 'COD_IDADE', 'IDADE',
       'DIAS_PERM', 'MORTE', 'NUM_FILHOS', 'INSTRU', 'RACA_COR', 'MARCA_UCI',
       'DIAGSEC1', 'DIAGSEC2', 'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 'DIAGSEC6',
       'DIAGSEC7', 'TPDISEC1', 'TPDISEC2', 'TPDISEC3', 'TPDISEC4', 'TPDISEC5',
       'TPDISEC6', 'TPDISEC7'],
      dtype='object')

Mostrar o dataframe após os descartes

In [12]:
df

,ESPEC,MUNIC_RES,SEXO,UTI_MES_TO,MARCA_UTI,QT_DIARIAS,DT_INTER,DT_SAIDA,DIAG_PRINC,COBRANCA,...,DIAGSEC5,DIAGSEC6,DIAGSEC7,TPDISEC1,TPDISEC2,TPDISEC3,TPDISEC4,TPDISEC5,TPDISEC6,TPDISEC7
0,01,430160,1,0,00,3,20201110,20201113,C498,12,...,None,None,None,0,0,0,0,0,0,0
1,02,430160,3,0,00,1,20201107,20201108,O809,61,...,None,None,None,0,0,0,0,0,0,0
2,02,430160,3,0,00,2,20201107,20201109,O664,61,...,None,None,None,0,0,0,0,0,0,0
3,02,430160,3,0,00,3,20201108,20201111,O664,61,...,None,None,None,0,0,0,0,0,0,0
4,02,430160,3,0,00,2,20201108,20201110,O664,61,...,None,None,None,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675598,03,430786,1,0,00,3,20200824,20200827,N219,12,...,None,None,None,0,0,0,0,0,0,0
675599,03,430786,3,0,00,3,20200824,20200827,K819,12,...,None,None,None,0,0,0,0,0,0,0
675600,03,430510,3,0,00,4,20200804,20200808,B342,12,...,None,None,None,0,0,0,0,0,0,0
675601,03,430595,1,0,00,5,20200731,20200805,A499,12,...,None,None,None,0,0,0,0,0,0,0


Formatar os dados para melhor visualização/leitura

In [13]:
if "UTI_MES_TO" in df.columns:
    df['UTI_MES_TO'] = df['UTI_MES_TO'].replace(UTI_MES_TO)
    df['UTI_MES_TO']

In [14]:
if "ESPEC" in df.columns:
    df['ESPEC'] = df['ESPEC'].replace(ESPEC)
    df['ESPEC']

In [15]:
if "DT_INTER" in df.columns:
    df['DT_INTER'] = df['DT_INTER'].apply(lambda x: pd.to_datetime(x).strftime('%Y/%m/%d'))
    df['DT_INTER']

In [16]:
if "DT_SAIDA" in df.columns:
    df['DT_SAIDA'] = df['DT_SAIDA'].apply(lambda x: pd.to_datetime(x).strftime('%Y/%m/%d'))
    df['DT_SAIDA']

In [17]:
if "COBRANCA" in df.columns:
    df['COBRANCA'] = df['COBRANCA'].replace(COBRANCA)
    df['COBRANCA']

In [18]:
if "COD_IDADE" in df.columns:
    df['COD_IDADE'] = df['COD_IDADE'].replace(COD_IDADE)
    df['COD_IDADE']

In [19]:
if "SEXO" in df.columns:
    df['SEXO'] = df['SEXO'].replace(SEXO)
    df['SEXO']

In [20]:
if "MORTE" in df.columns:
    df['MORTE'] = df['MORTE'].replace(MORTE)
    df['MORTE']

In [21]:
if "MARCA_UCI" in df.columns:
    df['MARCA_UCI'] = df['MARCA_UCI'].replace(MARCA_UCI)
    df['MARCA_UCI']

In [22]:
if "RACA_COR" in df.columns:
    df['RACA_COR'] = df['RACA_COR'].replace(RACA_COR)
    df['RACA_COR']

In [23]:
if "INSTRU" in df.columns:
    df['INSTRU'] = df['INSTRU'].replace(INSTRU)
    df['INSTRU']

In [24]:
if "MARCA_UTI" in df.columns:
    df['MARCA_UTI'] = df['MARCA_UTI'].replace(MARCA_UTI)
    df['MARCA_UTI']

In [25]:
if "DIAS_PERM" in df.columns:
    df['DIAS_PERM'] = df['DIAS_PERM'].replace(DIAS_PERM)
    df['DIAS_PERM']

In [26]:
if "CID_MORTE" in df.columns:
    df['CID_MORTE'] = df['CID_MORTE'].replace(CID)
    df['CID_MORTE']

In [27]:
if "DIAG_PRINC" in df.columns:
    df['DIAG_PRINC'] = df['DIAG_PRINC'].replace(CID)
    df['DIAG_PRINC']

In [28]:
if "DIAG_SECUN" in df.columns:
    df['DIAG_SECUN'] = df['DIAG_SECUN'].replace(CID)
    df['DIAG_SECUN']

In [29]:
if "DIAGSEC1" in df.columns:
    df['DIAGSEC1'] = df['DIAGSEC1'].replace(CID)

In [30]:
if "DIAGSEC2" in df.columns:
    df['DIAGSEC2'] = df['DIAGSEC2'].replace(CID)

In [31]:
if "DIAGSEC3" in df.columns:
    df['DIAGSEC3'] = df['DIAGSEC3'].replace(CID)

In [32]:
if "DIAGSEC4" in df.columns:
    df['DIAGSEC4'] = df['DIAGSEC4'].replace(CID)

In [33]:
if "DIAGSEC5" in df.columns:
    df['DIAGSEC5'] = df['DIAGSEC5'].replace(CID)

In [34]:
if "DIAGSEC6" in df.columns:
    df['DIAGSEC6'] = df['DIAGSEC6'].replace(CID)

In [35]:
if "DIAGSEC7" in df.columns:
    df['DIAGSEC7'] = df['DIAGSEC7'].replace(CID)

In [36]:
if "DIAGSEC8" in df.columns:
    df['DIAGSEC8'] = df['DIAGSEC8'].replace(CID)

In [37]:
if "DIAGSEC9" in df.columns:
    df['DIAGSEC9'] = df['DIAGSEC9'].replace(CID)

In [38]:
if "TPDISEC1" in df.columns:
    df['TPDISEC1'] = df['TPDISEC1'].replace(TPDISEC)

In [39]:
if "TPDISEC2" in df.columns:
    df['TPDISEC2'] = df['TPDISEC2'].replace(TPDISEC)

In [40]:
if "TPDISEC3" in df.columns:
    df['TPDISEC3'] = df['TPDISEC3'].replace(TPDISEC)

In [41]:
if "TPDISEC4" in df.columns:
    df['TPDISEC4'] = df['TPDISEC4'].replace(TPDISEC)

In [42]:
if "TPDISEC5" in df.columns:
    df['TPDISEC5'] = df['TPDISEC5'].replace(TPDISEC)

In [43]:
if "TPDISEC6" in df.columns:
    df['TPDISEC6'] = df['TPDISEC6'].replace(TPDISEC)

In [44]:
if "TPDISEC7" in df.columns:
    df['TPDISEC7'] = df['TPDISEC7'].replace(TPDISEC)

In [45]:
if "TPDISEC8" in df.columns:
    df['TPDISEC8'] = df['TPDISEC8'].replace(TPDISEC)

In [46]:
if "TPDISEC9" in df.columns:
    df['TPDISEC9'] = df['TPDISEC9'].replace(TPDISEC)

Renomear as colunas para melhor leitura

In [47]:
df.rename(columns=format_columns)
df.columns

Index(['ESPEC', 'MUNIC_RES', 'SEXO', 'UTI_MES_TO', 'MARCA_UTI', 'QT_DIARIAS',
       'DT_INTER', 'DT_SAIDA', 'DIAG_PRINC', 'COBRANCA', 'COD_IDADE', 'IDADE',
       'DIAS_PERM', 'MORTE', 'NUM_FILHOS', 'INSTRU', 'RACA_COR', 'MARCA_UCI',
       'DIAGSEC1', 'DIAGSEC2', 'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 'DIAGSEC6',
       'DIAGSEC7', 'TPDISEC1', 'TPDISEC2', 'TPDISEC3', 'TPDISEC4', 'TPDISEC5',
       'TPDISEC6', 'TPDISEC7'],
      dtype='object')

Mostrar o dataframe após limpeza

In [48]:
df

,ESPEC,MUNIC_RES,SEXO,UTI_MES_TO,MARCA_UTI,QT_DIARIAS,DT_INTER,DT_SAIDA,DIAG_PRINC,COBRANCA,...,DIAGSEC5,DIAGSEC6,DIAGSEC7,TPDISEC1,TPDISEC2,TPDISEC3,TPDISEC4,TPDISEC5,TPDISEC6,TPDISEC7
0,Cirúrgico,430160,Masculino,0,Não utilizou UTI,3,2020/11/10,2020/11/13,Neoplasia maligna do tecido conjuntivo e dos t...,Alta melhorado,...,None,None,None,,,,,,,
1,Obstétricos,430160,Feminino,0,Não utilizou UTI,1,2020/11/07,2020/11/08,Parto único espontâneo,Alta da mãe/puérpera e do recém-nascido,...,None,None,None,,,,,,,
2,Obstétricos,430160,Feminino,0,Não utilizou UTI,2,2020/11/07,2020/11/09,Falha de tentativa de trabalho de parto,Alta da mãe/puérpera e do recém-nascido,...,None,None,None,,,,,,,
3,Obstétricos,430160,Feminino,0,Não utilizou UTI,3,2020/11/08,2020/11/11,Falha de tentativa de trabalho de parto,Alta da mãe/puérpera e do recém-nascido,...,None,None,None,,,,,,,
4,Obstétricos,430160,Feminino,0,Não utilizou UTI,2,2020/11/08,2020/11/10,Falha de tentativa de trabalho de parto,Alta da mãe/puérpera e do recém-nascido,...,None,None,None,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675598,Clínico,430786,Masculino,0,Não utilizou UTI,3,2020/08/24,2020/08/27,Cálculo do trato urinário inferior,Alta melhorado,...,None,None,None,,,,,,,
675599,Clínico,430786,Feminino,0,Não utilizou UTI,3,2020/08/24,2020/08/27,K819,Alta melhorado,...,None,None,None,,,,,,,
675600,Clínico,430510,Feminino,0,Não utilizou UTI,4,2020/08/04,2020/08/08,Infecção por coronavírus de localização não es...,Alta melhorado,...,None,None,None,,,,,,,
675601,Clínico,430595,Masculino,0,Não utilizou UTI,5,2020/07/31,2020/08/05,Infecção bacteriana não especificada,Alta melhorado,...,None,None,None,,,,,,,


Exportar o dataframe limpo para o um .csv

In [49]:
df.to_csv(output_path, index=False)